In [1]:
#importing stuff and setting global variables
import cv2 as cv# for realtime processing
import math
import mediapipe as mp# fro AI 
#import pyautogui as py#yeh screenshot k liay hai
# from mediapipe_model_maker import gesture_recognizer

2023-12-03 23:39:23.014592: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 23:39:23.066692: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 23:39:23.279475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 23:39:23.279532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 23:39:23.318184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

The Code below i for creating the task or i would say a medol instance for recognizing guestures for a curstom data set
First piece of code is to load data from a folder into variable data and than split it into testing and training data
Next peice of code of is Training data than testing data data and finally exporting the model 

In [2]:
# IMAGES_PATH = "rps_data_sample"
# data = gesture_recognizer.Dataset.from_folder(
#     dirname=IMAGES_PATH,
#     hparams=gesture_recognizer.HandDataPreprocessingParams()
# )
# # Split the archive into training, validation and test dataset.
# train_data, rest_data = data.split(0.8)
# validation_data, test_data = rest_data.split(0.5)

In [3]:
# hparams = gesture_recognizer.HParams(export_dir="rock_paper_scissors_model")
# options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
# model = gesture_recognizer.GestureRecognizer.create(
#     train_data=train_data,
#     validation_data=validation_data,
#     options=options
# )

In [4]:
# loss, acc = model.evaluate(test_data, batch_size=1)
# print(f"Test loss:{loss}, Test accuracy:{acc}")

In [5]:
# model.export_model()

The code below is for drawing landmarks on real time frame

In [6]:
mp_holistic = mp.solutions.holistic

mp_drawing = mp.solutions.drawing_utils
def hand_detector(frame,holistic):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)

    ## for drawing left hand
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
        
    ## From drawing right hand
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
    
        
    return frame

The code bellow is load the model and use it for real-time and making it ready for real time camera application

In [7]:
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode
result_name = [None,0]
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
     #print('gesture recognition result: {}'.format(result))
     if len(result.gestures) and len(result.gestures[0]):
      gesture = result.gestures[0][0]
      if result_name[0] != gesture.category_name:
         result_name[0] = gesture.category_name
         result_name[1] = 0
      else:
         result_name[1] += 1
      print(gesture.category_name,result_name[1])
        
     else:
        result_name[0] = None
        result_name[1] = 0



    

options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='exported_model/gesture_recognizer.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,result_callback=print_result
    )
recognizer =  GestureRecognizer.create_from_options(options) 
timestamp = 0

I0000 00:00:1701628766.319474   14123 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701628766.323925   14183 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1701628766.331641   14123 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.8)
cap = cv.VideoCapture(0)

cap.set(cv.CAP_PROP_FRAME_HEIGHT,720)
cap.set(cv.CAP_PROP_FRAME_WIDTH,1080)
while cap.isOpened():
    timestamp += 1
    ret, frame = cap.read()
    if not ret:
        continue

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    recognition_result = recognizer.recognize_async(mp_image,timestamp)
    #print(result_name[0])

    frame = hand_detector(frame,holistic)

    cv.imshow("Hand Sign Detection", frame )    
    if cv.waitKey(1) & 0xFF == ord('q') or (result_name[0] == "scissors" and result_name[1] > 6):
        break

    if result_name[0] == "paper" and result_name[1] > 6:
        screenshot = py.screenshot()
        screenshot.save("screenshot.png")
    
        
cap.release()
cv.destroyAllWindows()


I0000 00:00:1701628766.442077   14123 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701628766.443724   14202 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


B 0
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
C 0
E 0
A 0
A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
A 13
A 14
A 15
A 16
A 17
A 18
A 19
A 20
A 21
A 22
A 23
A 24
A 25
A 26
A 27
A 28
A 29
A 30
A 31
A 32
N 0
K 0
K 1
K 2
K 3
K 4
K 5
K 6
K 7
K 8
K 9
K 10
K 11
K 12
K 13
K 14
K 15
K 16
K 17
K 18
K 19
K 20
K 21
K 22
K 23
K 24
K 25
K 26
K 27
O 0
C 0
C 1
C 2
C 3
C 4
C 5
C 6
C 7
C 8
C 9
C 10
C 11
C 12
C 13
C 14
C 15
C 16
C 17
C 18
C 19
C 20
C 21
C 22
C 23
C 24
C 25
C 26
C 27
C 28
C 29
C 30
C 31
C 32
C 33
C 34
C 35
C 36
C 37
C 38
C 39
C 40
C 41
C 42
C 43
C 44
C 45
C 46
C 47
C 48
C 49
C 50
C 51
C 52
C 53
C 54
B 0
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
B 27
B 28
B 29
B 30
B 31
B 32
B 33
B 34
B 35
B 36
B 37
O 0
B 0
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
